In [2]:
# Run MPNN on the inpainted regions

import sys

sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import gen_array_tasks

design_list_file = "/home/broerman/crispy_shifty/projects/DAB/round_1/design/01_inpaint_structure/inpainted_structures.list"

simulation_name = '02_mpnn_structure'
output_path = f"/home/broerman/crispy_shifty/projects/DAB/round_1/design/{simulation_name}"

options = " ".join(
    [
        "out:level 100",
    ]
)
extra_kwargs = {
    "selection_name": "inpaint_seq_resis", 
    "df_scores": "/home/broerman/crispy_shifty/projects/DAB/round_1/design/01_inpaint_structure/scores.csv",
    "mpnn_design_area": "selection", 
    "num_sequences": "8", 
    "batch_size": "8",
    "omit_AAs": "CX",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_selection",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="cpu",
    memory="5G",
    time="00:10:00",
    nstruct=1,
    nstruct_per_task=10,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
    sha1=None
)

Run the following command with your desired environment active:
sbatch -a 1-18 /home/broerman/crispy_shifty/projects/DAB/round_1/design/02_mpnn_structure/run.sh


In [1]:
import os
import sys

sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = '02_mpnn_structure'
output_path = f"/home/broerman/crispy_shifty/projects/DAB/round_1/design/{simulation_name}"

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

https://docs.anaconda.com/anaconda/install



In [2]:
from crispy_shifty.utils.io import parse_scorefile_linear

scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
# scores_df = scores_df.convert_dtypes()

  0%|          | 0/180 [00:00<?, ?it/s]

In [3]:
import numpy as np
scores_df["designed_by"] = np.where(scores_df["designed_by"] == "mpnn", "mpnn", "inpainting")

KeyError: 'designed_by'

In [4]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

  0%|          | 0/180 [00:00<?, ?it/s]

In [5]:
from tqdm import tqdm

scores_df = scores_df.sort_values("total_length")

with open(os.path.join(output_path, "mpnn_inpaints.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

100%|██████████| 180/180 [00:00<00:00, 612307.15it/s]


In [6]:
with open(os.path.join(output_path, "mpnn_inpaints.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

100%|██████████| 180/180 [00:00<00:00, 169276.84it/s]
